# pre works

In [ ]:
# -- 필요한 라이브러리 설치 --
!pip install transformers
!pip install Keras-Preprocessing
# !pip install fasttext

In [ ]:
# -- import --
import pandas as pd
import torch
from transformers import ElectraForSequenceClassification, AutoTokenizer, AdamW, ElectraTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import random
import numpy as np
from sklearn.metrics import classification_report
# GPU 사용
device = torch.device("cuda")

In [11]:
# # -- making temporary df to test multimodal --
# df_for_multimodal = pd.DataFrame()
# for cat_id in df['cat_id'].unique():
#     df_for_multimodal = pd.concat([df_for_multimodal, df[df['cat_id'] == cat_id].head(1)])
# df_for_multimodal = df_for_multimodal.reset_index(drop=True)
# df_for_multimodal.to_csv('/content/drive/MyDrive/df_for_multimodal.csv', index=False)

# -- 필요 데이터 --
import pandas as pd
import re

# -- 간단한 테스트용 170개 row 데이터 --
# df = pd.read_csv('/content/drive/MyDrive/df_for_multimodal.csv')
# df['product_name'] = df['product_name'].apply(lambda x : re.sub('[^가-힣\sa-zA-Z]', '', x))
# df.info()

# -- 실제 이미지 크롤링 할 때 사용한 파일 --
df = pd.read_csv('/content/drive/MyDrive/bungae_df_for_multimodal')
df['product_id'] = df['product_id'].astype(str)
df['cat_id'] = df['cat_id'].astype(str)
df['main_cat'] = df['main_cat'].astype(str)
df['mid_cat'] = df['mid_cat'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98360 entries, 0 to 98359
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    98360 non-null  object 
 1   product_name  98360 non-null  object 
 2   image_url     98360 non-null  object 
 3   image_cnt     98360 non-null  float64
 4   cat_id        98360 non-null  object 
 5   main_cat      98360 non-null  object 
 6   mid_cat       98360 non-null  object 
dtypes: float64(1), object(6)
memory usage: 5.3+ MB


In [ ]:
# -- 다시 필요한 작업 --
# 1. 이미지를 크롤링 하면서 사라진 것들이 있음. 이미지가 없는 row는 삭제해주기
# 2. 메인 카테고리로 할 것이기 때문에 이미지 폴더 대분류 기준으로 다시 만들고 이미지들도 그에 맞게 옮겨주기

# --> 주피터 노트북에서 작업

In [ ]:
# -- 업로드 완료된 이미지 폴더 메인 카테고리 기준으로 나누기 --
import os

# 이미지 폴더 다시 만들기
os.makedirs(f'/content/drive/MyDrive/bungae_image_folder_multimodal')

In [ ]:
# 메인 카테고리 번호 기준으로 파일 옮기기
import shutil
import os

# 옮기기 전 경로
file_source1 = '/content/drive/MyDrive/310'
file_source2 = '/content/drive/MyDrive/400_320'
file_source3 = '/content/drive/MyDrive/430_420_405'

# 최종 경로
file_destination = '/content/drive/MyDrive/bungae_image_folder_multimodal'

# 폴더 안 폴더 파일들
# source_folder1 = os.listdir(file_source1)
source_folder2 = os.listdir(file_source2)
source_folder3 = os.listdir(file_source3)

destination_folder = os.listdir(file_destination)

# 파일 옮기기

# shutil.move(file_source1, file_destination)
for f in (source_folder2 + source_folder3):
    if len(f) == 3:
        if f in source_folder2:
            shutil.move(file_source2 + '/' + f, file_destination)
        elif f in source_folder3:
            shutil.move(file_source3 + '/' + f, file_destination)

In [ ]:
# -- 한 폴더 안에 파일이 너무 많으면 구글 코랩에서 찾는 시간으로 인해
# input/output error가 생김 --
# -- 때문에 불가피하게 소분류부터 (한 폴더당 600개) 분류하는 모델을 먼저 만들어보기 위한 셋업 --

In [14]:
# -- 소분류 기준으로 파일 새로 만들기 --
import os


def make_folders_for_image():
    cat_list = list(df['cat_id'].unique())

    for cat_id in cat_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'/content/drive/MyDrive/bungae_image_folder_600_each/{cat_id}')

    return

make_folders_for_image()

In [ ]:
# -- 중간에 안 넘어온 파일들이 있음 --
# -- 다시 넘겨주기 --

In [23]:
# -- 이미지 파일들 소분류로 다시 대이동 --
import shutil
import os
from tqdm.notebook import tqdm

base_path = '/content/drive/MyDrive/bungae_image_folder_multimodal'
destination_path = '/content/drive/MyDrive/bungae_image_folder_600_each' # 폴더당 약 600개의 파일이 있을 예정 (더 적은 경우도 있음)
for idx in tqdm(range(len(df))):
    main_cat = df.loc[idx, 'main_cat']
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']

    # 대분류로 분류되어있던 이미지 경로
    image_path = f"{base_path}/{main_cat}/{prd_id}_image.jpg"
    if os.path.isfile(image_path): # 아직 안넘어 왔을 경우

        # 소분류로 분류될 최종 경로 (이미지는 뒤에 경로로 추가하지 않음)
        desti_path = f"{destination_path}/{cat_id}"
        shutil.move(image_path, desti_path)

  0%|          | 0/98360 [00:00<?, ?it/s]

In [24]:
# -- 실수로 잘못 옮겨서 다시 옮겨야 됨 ㅠ --
import shutil
import os
from tqdm.notebook import tqdm

base_path = '/content/drive/MyDrive/bungae_image_folder_600_each'
for idx in tqdm(range(len(df))):
    main_cat = df.loc[idx, 'main_cat']
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']

    image_path = f"{base_path}/{prd_id}_image.jpg"

    if os.path.isfile(image_path): # 아직 안 넘어왔을 경우

        # 소분류로 분류될 최종 경로 (이미지는 뒤에 경로로 추가하지 않음)
        desti_path = f"{base_path}/{cat_id}"
        shutil.move(image_path, desti_path)

  0%|          | 0/98360 [00:00<?, ?it/s]

In [25]:
# -- 파일이 다 넘어왔는지 확인하는 코드 --
# -- 파일 전체 개수 다 더해서 -> 98360가 나와야 됨 --
base_path = '/content/drive/MyDrive/bungae_image_folder_600_each'

total_files_count = 0
for folder in os.listdir(base_path):
    total_files_count += len(os.listdir(base_path + '/' + folder))

print(total_files_count)

98359


In [27]:
# -- 파일 개수가 98360이 나와야 하는데 98359가 나옴 -> 확인 필요 --
# -- '212297168'가 없음 --
for idx in tqdm(range(len(df))):
    main_cat = df.loc[idx, 'main_cat']
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']

    image_path = f"{base_path}/{cat_id}/{prd_id}_image.jpg"

    if os.path.isfile(image_path) == False:
        print(prd_id)


  0%|          | 0/98360 [00:00<?, ?it/s]

212297168


In [32]:
df[df['product_id'] == '212297168']['image_url'].values

array(['https://media.bunjang.co.kr/product/212297168_{cnt}_1676203539_w{res}.jpg'],
      dtype=object)

In [33]:
# -- 하나만 따로 빼서 이미지 크롤링 후 폴더에 넣기 --

import urllib.request
import time
import pandas as pd
import re
from PIL import Image
import gc

cat_id = '320080700'
prd_id = '212297168'
target_size=(224, 224)
url = 'https://media.bunjang.co.kr/product/212297168_{cnt}_1676203539_w{res}.jpg'

urllib.request.urlretrieve(url, f'/content/drive/MyDrive/bungae_image_folder_600_each/{cat_id}/{prd_id}_image.jpg')
image_path = f'/content/drive/MyDrive/bungae_image_folder_600_each/{cat_id}/{prd_id}_image.jpg'
image = Image.open(image_path)
image = image.resize(target_size)
# Save the resized image
image.save(image_path)


In [34]:
# -- 파일이 다 넘어왔는지 다시 확인하는 코드 --

base_path = '/content/drive/MyDrive/bungae_image_folder_600_each'

total_files_count = 0
for folder in os.listdir(base_path):
    total_files_count += len(os.listdir(base_path + '/' + folder))

print(total_files_count)

98360


In [ ]:
# -- 이미지 파일 가져오기 끝 --

# 파일 카피 만드는 과정

In [ ]:
!sudo chmod a+r /content/drive/MyDrive/bungae_image_folder_multimodal/320/177404650_image.jpg

In [ ]:
!ls -l /content/drive/MyDrive/bungae_image_folder_multimodal/320/177404650_image.jpg

-rw------- 1 root root 5334 Jun 14 02:04 /content/drive/MyDrive/bungae_image_folder_multimodal/320/177404650_image.jpg


In [ ]:
# -- 디렉토리 삭제 --
import shutil
shutil.rmtree('/content/drive/MyDrive/bungae_image_folder_multimodal_copy')

In [ ]:
# -- copy본을 옮길 파일 다시 만들기 --
import os
import pandas as pd

def make_folders_for_copy_image():
    df = pd.read_csv('/content/drive/MyDrive/bungae_df_for_multimodal')
    cat_id_list = list(df['main_cat'].unique())

    for cat_id in cat_id_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'/content/drive/MyDrive/bungae_image_folder_multimodal_copy/{cat_id}')

    return

make_folders_for_copy_image()

In [ ]:
# -- 이미지 카피본 만들어서 옮기기 --
import os
import shutil
from tqdm.notebook import tqdm

file_source = '/content/drive/MyDrive/bungae_image_folder_multimodal'

get_file_sources = os.listdir(file_source)
get_file_sources

# -- base code --
# import shutil

# # Source and destination paths
# src_path = '/content/drive/MyDrive/bungae_image_folder_multimodal/320/177404650_image.jpg'
# dst_path = '/content/copy_image.jpg'

# # Make a copy of the image file
# shutil.copy(src_path, dst_path)



['310', '430', '420', '405', '320', '400']

In [ ]:
# -- 나누어서 하기 --
# 310
folder_310 = get_file_sources[0]
file_source2 = file_source + '/' + folder_310
get_file_sources2 = os.listdir(file_source2)

for g2 in tqdm(get_file_sources2):
    src_path = file_source2 + '/' + g2
    dst_path = '/content/drive/MyDrive/bungae_image_folder_multimodal_copy' + '/' + folder_310 + '/' + 'copy_' + g2
    shutil.copy(src_path, dst_path)
    # change permissions
    os.chmod(dst_path, 0o644)

  0%|          | 0/37287 [00:00<?, ?it/s]

In [ ]:
# -- 나누어서 하기 --
# 430
folder_430 = get_file_sources[1]
file_source2 = file_source + '/' + folder_430
get_file_sources2 = os.listdir(file_source2)

for g2 in tqdm(get_file_sources2):
    src_path = file_source2 + '/' + g2
    dst_path = '/content/drive/MyDrive/bungae_image_folder_multimodal_copy' + '/' + folder_430 + '/' + 'copy_' + g2
    shutil.copy(src_path, dst_path)
    # change permissions
    os.chmod(dst_path, 0o644)

  0%|          | 0/8399 [00:00<?, ?it/s]

In [ ]:
# -- 나누어서 하기 --
# 420
folder_420 = get_file_sources[2]
file_source2 = file_source + '/' + folder_420
get_file_sources2 = os.listdir(file_source2)

for g2 in tqdm(get_file_sources2):
    src_path = file_source2 + '/' + g2
    dst_path = '/content/drive/MyDrive/bungae_image_folder_multimodal_copy' + '/' + folder_420 + '/' + 'copy_' + g2
    shutil.copy(src_path, dst_path)
    # change permissions
    os.chmod(dst_path, 0o644)

  0%|          | 0/4800 [00:00<?, ?it/s]

In [ ]:
# -- 나누어서 하기 --
# 405
folder_405 = get_file_sources[3]
file_source2 = file_source + '/' + folder_405
get_file_sources2 = os.listdir(file_source2)

for g2 in tqdm(get_file_sources2):
    src_path = file_source2 + '/' + g2
    dst_path = '/content/drive/MyDrive/bungae_image_folder_multimodal_copy' + '/' + folder_405 + '/' + 'copy_' + g2
    shutil.copy(src_path, dst_path)
    # change permissions
    os.chmod(dst_path, 0o644)

  0%|          | 0/6000 [00:00<?, ?it/s]

In [ ]:
# -- 나누어서 하기 --
# 320
folder_320 = get_file_sources[4]
file_source2 = file_source + '/' + folder_320
get_file_sources2 = os.listdir(file_source2)

for g2 in tqdm(get_file_sources2):
    src_path = file_source2 + '/' + g2
    dst_path = '/content/drive/MyDrive/bungae_image_folder_multimodal_copy' + '/' + folder_320 + '/' + 'copy_' + g2
    shutil.copy(src_path, dst_path)
    # change permissions
    os.chmod(dst_path, 0o644)

  0%|          | 0/29874 [00:00<?, ?it/s]

In [ ]:
# -- 나누어서 하기 --
# 400
folder_400 = get_file_sources[5]
file_source2 = file_source + '/' + folder_400
get_file_sources2 = os.listdir(file_source2)

for g2 in tqdm(get_file_sources2):
    src_path = file_source2 + '/' + g2
    dst_path = '/content/drive/MyDrive/bungae_image_folder_multimodal_copy' + '/' + folder_400 + '/' + 'copy_' + g2
    shutil.copy(src_path, dst_path)
    # change permissions
    os.chmod(dst_path, 0o644)

  0%|          | 0/12000 [00:00<?, ?it/s]

In [ ]:
import subprocess

dst_path = "/content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg"

# Change the file permissions using the shell command
subprocess.run(["chmod", "644", dst_path])

CompletedProcess(args=['chmod', '644', '/content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg'], returncode=0)

In [ ]:
os.chmod('/content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg', 0o644)

In [ ]:
!ls -l /content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg

-rw------- 1 root root 5334 Jun 15 03:31 /content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg


In [ ]:
%%shell

chmod 644 /content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg

In [ ]:
!ls -l /content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg

-rw------- 1 root root 5334 Jun 15 03:31 /content/drive/MyDrive/bungae_image_folder_multimodal_copy/320/copy_177404650_image.jpg


In [ ]:
# -- 새로운 파일 만들어서 옮기기 --


# ----- main below ------

# Multimodal test model (Fasttext + MobileNet2) -> failed

In [ ]:
# -- model --
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import mobilenet_v2
import fasttext
from sklearn.preprocessing import LabelEncoder
import gensim


# Define the multi-modal model
class MultiModalModel(nn.Module):
    def __init__(self, num_classes):
        super(MultiModalModel, self).__init__()

        # Load the FastText word embedding model
        self.word_embedding = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/wiki.ko.vec', binary=False)

        # Load the MobileNetV2 pre-trained model
        self.image_model = mobilenet_v2(pretrained=True)
        self.image_model.classifier = nn.Identity()

        # Text processing layers
        self.text_fc = nn.Linear(300, 128)

        # Image processing layers
        self.image_fc = nn.Linear(1280, 128)

        # Output layer
        self.output_fc = nn.Linear(256, num_classes)

    def forward(self, texts, images):
        # Text processing
        if isinstance(texts, tuple):
            # Handle batch input
            text_tokens = texts[0].split()
            text_embedding = torch.mean(
                torch.stack([torch.from_numpy(self.word_embedding.get_vector(token))
                            if token in self.word_embedding.key_to_index
                            else torch.zeros(300)
                            for token in text_tokens]), dim=0)
        else:
            # Handle single input
            text_tokens = texts.split()
            text_embedding = torch.mean(
                torch.stack([torch.from_numpy(self.word_embedding.get_vector(token))
                            if token in self.word_embedding.key_to_index
                            else torch.zeros(300)
                            for token in text_tokens]), dim=0)

        text_output = F.relu(self.text_fc(text_embedding))

        # Image processing
        image_output = self.image_model(images)

        # Combine modalities
        combined = torch.cat((text_output, image_output), dim=1)

        # Classification
        output = self.classifier(combined)
        return output

# Custom Dataset class
class MultiModalDataset(Dataset):
    def __init__(self, texts, images, labels):
        self.texts = texts
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        text = self.texts[index]
        image = self.images[index]
        label = self.labels[index]
        return text, image, label


In [ ]:
# -- text data --
texts = list(df['product_name'])

# -- image paths --

images = list()
for idx in range(len(df)):
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']
    image_path = f'data/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg'
    images.append(image_path)

# -- label --
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['cat_id'])
labels = torch.tensor(list(df['label']))

# Create the multi-modal dataset and data loader
dataset = MultiModalDataset(texts, images, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
# Initialize the model
num_classes = 170
model = MultiModalModel(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from tqdm.notebook import tqdm

# Training loop
for epoch in range(10):
    for texts, images, labels in dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts, images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print the loss for monitoring
        print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')

# Multimodal test model (Koelectra + MobileNet2) -> working

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraModel
from PIL import Image


# Custom Dataset
class MultiModalDataset(Dataset):
    def __init__(self, text_data, image_data, labels, text_tokenizer):
        self.text_data = text_data
        self.image_data = image_data
        self.labels = labels
        self.text_tokenizer = text_tokenizer
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        image = Image.open(self.image_data[idx]).convert("RGB")
        image = self.transform(image)
        label = self.labels[idx]

        return text, image, label

# Define your data examples (text, image paths, and labels)
text_data = list(df['product_name'])

image_data = list()
for idx in range(len(df)):
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']
    image_path = f'data/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg'
    image_data.append(image_path)

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['cat_id'])
labels = df['label']

# Initialize the text tokenizer
text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")


# Create the custom dataset
dataset = MultiModalDataset(text_data, image_data, labels, text_tokenizer)

# Define data loaders
batch_size = 8
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the multi-modal model
class MultiModalModel(nn.Module):
    def __init__(self, num_classes):
        super(MultiModalModel, self).__init__()

        # Text Model (KoElectra)
        self.text_model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

        # Image Model (MobileNetV2)
        self.image_model = models.mobilenet_v2(pretrained=True)
        self.image_model.classifier = nn.Identity()  # Remove the final fully connected layer

        # Fusion Layer
        fusion_dim = self.text_model.config.hidden_size + self.image_model.last_channel
        self.fusion_layer = nn.Linear(fusion_dim, fusion_dim)

        # Output Layer
        self.output_layer = nn.Linear(fusion_dim, num_classes)

    def forward(self, text_inputs, image_inputs):
        # Text Encoding
        text_input_ids = self.text_tokenizer.batch_encode_plus(
            text_inputs,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )["input_ids"]
        text_outputs = self.text_model(input_ids=text_input_ids)[0][:, 0, :]  # Use the CLS token embedding

        # Image Encoding
        image_outputs = self.image_model.features(image_inputs)
        image_outputs = torch.nn.functional.adaptive_avg_pool2d(image_outputs, 1).reshape(image_outputs.size(0), -1)

        # Fusion
        fusion_inputs = torch.cat((text_outputs, image_outputs), dim=1)
        fused_outputs = self.fusion_layer(fusion_inputs)

        # Output
        logits = self.output_layer(fused_outputs)

        return logits



In [ ]:
# Initialize the multi-modal model
num_classes = 170  # Specify the number of output classes
model = MultiModalModel(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = criterion.to(device)

for epoch in range(num_epochs):
    for texts, images, labels in data_loader:
        texts = list(texts)
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts, images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# After training, you can use the model for predictions on new data

# Multimodal (KoElectra + MobileNet2) -> train, val, and test

In [ ]:
# -- 필요한 라이브러리 설치 --
!pip install transformers
!pip install Keras-Preprocessing

In [2]:
# -- 필요한 데이터 --
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/bungae_df_for_multimodal')
df.head()

,product_id,product_name,image_url,image_cnt,cat_id,main_cat,mid_cat
0,217365207,경량롱패딩,https://media.bunjang.co.kr/product/217365207_...,3.0,310090050,310,310090
1,175507846,롱패딩 판매합니다,https://media.bunjang.co.kr/product/175507846_...,4.0,310090050,310,310090
2,213094512,피레넥스 여성 화이트 패딩 55판-69,https://media.bunjang.co.kr/product/213094512_...,10.0,310090050,310,310090
3,208884568,보브 다운패딩,https://media.bunjang.co.kr/product/208884568_...,2.0,310090050,310,310090
4,164395818,에이글 코트타입 구스 자켓 L사이즈,https://media.bunjang.co.kr/product/164395818_...,5.0,310090050,310,310090


In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraModel
from torchvision.models import mobilenet_v2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

# Custom Dataset
class MultiModalDataset(Dataset):
    def __init__(self, text_data, image_data, labels, text_tokenizer, transform):
        self.text_data = text_data
        self.image_data = image_data
        self.labels = labels
        self.text_tokenizer = text_tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        image = Image.open(self.image_data[idx]).convert("RGB")
        image = self.transform(image)
        label = self.labels.iloc[idx]  # Use `iloc` to access the label based on the index

        return text, image, label

# Split data into train, validation, and test sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_data, test_size=0.3, random_state=42)

# Define your data examples (text, image paths, and labels)
text_train_data = list(train_data['product_name'])
text_val_data = list(val_data['product_name'])
text_test_data = list(test_data['product_name'])

image_train_data = list()
image_val_data = list()
image_test_data = list()
for idx in range(len(df)):
    cat_id = df.loc[idx, 'cat_id'] # 소분류
    prd_id = df.loc[idx, 'product_id']
    # -- 카테고리 분류에 따라 image_path 조정 필요 --
    image_path = f'/content/drive/MyDrive/bungae_image_folder_600_each/{cat_id}/{prd_id}_image.jpg'
    if idx in train_data.index:
        image_train_data.append(image_path)
    elif idx in val_data.index:
        image_val_data.append(image_path)
    else:
        image_test_data.append(image_path)

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['cat_id'])
labels_train = df.loc[train_data.index, 'label']
labels_val = df.loc[val_data.index, 'label']
labels_test = df.loc[test_data.index, 'label']

In [4]:
# Initialize the text tokenizer
text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

# Define transformations for image data
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create the custom datasets
train_dataset = MultiModalDataset(text_train_data, image_train_data, labels_train, text_tokenizer, image_transform)
val_dataset = MultiModalDataset(text_val_data, image_val_data, labels_val, text_tokenizer, image_transform)
test_dataset = MultiModalDataset(text_test_data, image_test_data, labels_test, text_tokenizer, image_transform)

# Define data loaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Define the multi-modal model
class MultiModalModel(nn.Module):
    def __init__(self, num_classes):
        super(MultiModalModel, self).__init__()

        # Text Model (KoElectra)
        self.text_model = ElectraModel.from_pretrained("monologg/koelectra-small-v3-discriminator")
        self.text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

        # Image Model (MobileNetV2)
        self.image_model = mobilenet_v2(pretrained=True)
        self.image_model.classifier = nn.Identity()  # Remove the final fully connected layer

        # Fusion Layer
        fusion_dim = self.text_model.config.hidden_size + self.image_model.last_channel
        self.fusion_layer = nn.Linear(fusion_dim, fusion_dim)

        # Output Layer
        self.output_layer = nn.Linear(fusion_dim, num_classes)

    def forward(self, text_inputs, image_inputs):
        # Text Encoding
        text_input_ids = self.text_tokenizer.batch_encode_plus(
            text_inputs,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )["input_ids"]
        text_outputs = self.text_model(input_ids=text_input_ids)[0][:, 0, :]  # Use the CLS token embedding

        # Image Encoding
        image_outputs = self.image_model.features(image_inputs)
        image_outputs = torch.nn.functional.adaptive_avg_pool2d(image_outputs, 1).reshape(image_outputs.size(0), -1)

        # Fusion
        fusion_inputs = torch.cat((text_outputs, image_outputs), dim=1)
        fused_outputs = self.fusion_layer(fusion_inputs)

        # Output
        logits = self.output_layer(fused_outputs)

        return logits

In [6]:
# Initialize the multi-modal model
num_classes = 170  # main_cat에 대해서만
model = MultiModalModel(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = criterion.to(device)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in tqdm(range(num_epochs)):
    # Training
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for texts, images, labels in train_loader:
        texts = list(texts)
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts, images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100.0 * train_correct / train_total

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for texts, images, labels in val_loader:
            texts = list(texts)
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(texts, images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100.0 * val_correct / val_total

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Print the loss and accuracy for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/l

  0%|          | 0/10 [00:00<?, ?it/s]

KeyError: ignored

# Multimodal with data augumentation and dropout (KoElectra + MobileNet2)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import ElectraTokenizer, ElectraModel
from PIL import Image


# Custom Dataset
class MultiModalDataset(Dataset):
    def __init__(self, text_data, image_data, labels, text_tokenizer, transform=None):
        self.text_data = text_data
        self.image_data = image_data
        self.labels = labels
        self.text_tokenizer = text_tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        image = Image.open(self.image_data[idx]).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        return text, image, label


# Define your data examples (text, image paths, and labels)
text_data = list(df['product_name'])

image_data = list()
for idx in range(len(df)):
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']
    image_path = f'data/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg'
    image_data.append(image_path)

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['cat_id'])
labels = df['label']

# Initialize the text tokenizer
text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# Define data augmentation transforms
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create the custom dataset with data augmentation
dataset = MultiModalDataset(text_data, image_data, labels, text_tokenizer, transform=image_transform)


In [ ]:
# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Define data loaders for train, validation, and test sets
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Define the multi-modal model
class MultiModalModel(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super(MultiModalModel, self).__init__()

        # Text Model (KoElectra)
        self.text_model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

        # Image Model (MobileNetV2)
        self.image_model = models.mobilenet_v2(pretrained=True)
        self.image_model.classifier = nn.Identity()  # Remove the final fully connected layer

        # Fusion Layer
        fusion_dim = self.text_model.config.hidden_size + self.image_model.last_channel
        self.fusion_layer = nn.Linear(fusion_dim, fusion_dim)

        # Output Layer
        self.output_layer = nn.Linear(fusion_dim, num_classes)

        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text_inputs, image_inputs):
        # Text Encoding
        text_input_ids = self.text_tokenizer.batch_encode_plus(
            text_inputs,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )["input_ids"]
        text_outputs = self.text_model(input_ids=text_input_ids)[0][:, 0, :]  # Use the CLS token embedding

        # Image Encoding
        image_outputs = self.image_model.features(image_inputs)
        image_outputs = torch.nn.functional.adaptive_avg_pool2d(image_outputs, 1).reshape(image_outputs.size(0), -1)

        # Fusion
        fusion_inputs = torch.cat((text_outputs, image_outputs), dim=1)
        fused_outputs = self.fusion_layer(fusion_inputs)

        # Dropout
        fused_outputs = self.dropout(fused_outputs)

        # Output
        logits = self.output_layer(fused_outputs)

        return logits


# Initialize the multi-modal model
num_classes = 170  # Specify the number of output classes
model = MultiModalModel(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = criterion.to(device)

for epoch in range(num_epochs):
    train_loss = []
    train_correct = 0
    train_total = 0

    model.train()

    for texts, images, labels in train_loader:
        texts = list(texts)
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts, images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track training loss
        train_loss.append(loss.item())

        # Track training accuracy
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()

    # Calculate average training loss
    avg_train_loss = sum(train_loss) / len(train_loss)

    # Calculate training accuracy
    train_accuracy = 100.0 * train_correct / train_total

    # Validation loop
    val_loss = []
    val_correct = 0
    val_total = 0

    model.eval()

    with torch.no_grad():
        for texts, images, labels in val_loader:
            texts = list(texts)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(texts, images)
            loss = criterion(outputs, labels)

            # Track validation loss
            val_loss.append(loss.item())

            # Track validation accuracy
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    # Calculate average validation loss
    avg_val_loss = sum(val_loss) / len(val_loss)

    # Calculate validation accuracy
    val_accuracy = 100.0 * val_correct / val_total

    # Print the loss and accuracy for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# After training, you can use the model for predictions on new data


# with SMOTE

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import ElectraTokenizer, ElectraModel
from PIL import Image
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# Custom Dataset
class MultiModalDataset(Dataset):
    def __init__(self, text_data, image_data, labels, text_tokenizer, transform=None):
        self.text_data = text_data
        self.image_data = image_data
        self.labels = labels
        self.text_tokenizer = text_tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        image = Image.open(self.image_data[idx]).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        return text, image, label


# Define your data examples (text, image paths, and labels)
text_data = list(df['product_name'])

image_data = list()
for idx in range(len(df)):
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']
    image_path = f'data/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg'
    image_data.append(image_path)

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['cat_id'])
labels = df['label']

# Apply SMOTE for oversampling
oversampler = SMOTE()
text_data, image_data, labels = oversampler.fit_resample(text_data, image_data, labels)

# Apply RandomUnderSampler for undersampling
undersampler = RandomUnderSampler()
text_data, image_data, labels = undersampler.fit_resample(text_data, image_data, labels)

# Initialize the text tokenizer
text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# Define data augmentation transforms
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create the custom dataset with data augmentation
dataset = MultiModalDataset(text_data, image_data, labels, text_tokenizer, transform=image_transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Define data loaders for train, validation, and test sets
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the multi-modal model
class MultiModalModel(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super(MultiModalModel, self).__init__()

        # Text Model (KoElectra)
        self.text_model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

        # Image Model (MobileNetV2)
        self.image_model = models.mobilenet_v2(pretrained=True)
        self.image_model.classifier = nn.Identity()  # Remove the final fully connected layer

        # Fusion Layer
        fusion_dim = self.text_model.config.hidden_size + self.image_model.last_channel
        self.fusion_layer = nn.Linear(fusion_dim, fusion_dim)

        # Output Layer
        self.output_layer = nn.Linear(fusion_dim, num_classes)

        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text_inputs, image_inputs):
        # Text Encoding
        text_input_ids = self.text_tokenizer.batch_encode_plus(
            text_inputs,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )["input_ids"]
        text_outputs = self.text_model(input_ids=text_input_ids)[0][:, 0, :]  # Use the CLS token embedding

        # Image Encoding
        image_outputs = self.image_model.features(image_inputs)
        image_outputs = torch.nn.functional.adaptive_avg_pool2d(image_outputs, 1).reshape(image_outputs.size(0), -1)

        # Fusion
        fusion_inputs = torch.cat((text_outputs, image_outputs), dim=1)
        fused_outputs = self.fusion_layer(fusion_inputs)

        # Dropout
        fused_outputs = self.dropout(fused_outputs)

        # Output
        logits = self.output_layer(fused_outputs)

        return logits


# Initialize the multi-modal model
num_classes = 170  # Specify the number of output classes
model = MultiModalModel(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = criterion.to(device)

for epoch in range(num_epochs):
    train_loss = []
    train_correct = 0
    train_total = 0

    model.train()

    for texts, images, labels in train_loader:
        texts = list(texts)
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts, images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track training loss
        train_loss.append(loss.item())

        # Track training accuracy
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()

    # Calculate average training loss
    avg_train_loss = sum(train_loss) / len(train_loss)

    # Calculate training accuracy
    train_accuracy = 100.0 * train_correct / train_total

    # Validation loop
    val_loss = []
    val_correct = 0
    val_total = 0

    model.eval()

    with torch.no_grad():
        for texts, images, labels in val_loader:
            texts = list(texts)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(texts, images)
            loss = criterion(outputs, labels)

            # Track validation loss
            val_loss.append(loss.item())

            # Track validation accuracy
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    # Calculate average validation loss
    avg_val_loss = sum(val_loss) / len(val_loss)

    # Calculate validation accuracy
    val_accuracy = 100.0 * val_correct / val_total

    # Print the loss and accuracy for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# After training, you can use the model for predictions on new data


In [ ]:
# -- version 2 --

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import ElectraTokenizer, ElectraModel
from PIL import Image
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# Custom Dataset
class MultiModalDataset(Dataset):
    def __init__(self, text_data, image_data, labels, text_tokenizer, transform=None):
        self.text_data = text_data
        self.image_data = image_data
        self.labels = labels
        self.text_tokenizer = text_tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        image = Image.open(self.image_data[idx]).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]

        return text, image, label


# Define your data examples (text, image paths, and labels)
text_data = list(df['product_name'])
image_data = list()

for idx in range(len(df)):
    cat_id = df.loc[idx, 'cat_id']
    prd_id = df.loc[idx, 'product_id']
    image_path = f'data/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg'
    image_data.append(image_path)

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['cat_id'])
labels = df['label']

# Initialize the text tokenizer
text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# Define data augmentation transforms
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create the custom dataset with data augmentation
dataset = MultiModalDataset(text_data, image_data, labels, text_tokenizer, transform=image_transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Apply SMOTE and RandomUnderSampler to balance the dataset
smote = SMOTE(sampling_strategy='auto', random_state=42)
under_sampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

train_text = [sample[0] for sample in train_dataset]
train_images = torch.stack([sample[1] for sample in train_dataset])
train_labels = torch.tensor([sample[2] for sample in train_dataset])

train_text_reshape = [text.reshape(-1, 1) for text in train_text]
train_text_resampled, train_labels_resampled = smote.fit_resample(torch.cat(train_text_reshape), train_labels)
train_text_resampled, train_labels_resampled = under_sampler.fit_resample(train_text_resampled, train_labels_resampled)

train_text_resampled = [text.flatten().tolist() for text in train_text_resampled]
train_labels_resampled = train_labels_resampled.tolist()

# Convert the resampled data back to tensors
train_text_resampled = [torch.tensor(text) for text in train_text_resampled]
train_text_resampled = [text.squeeze() for text in train_text_resampled]
train_text_resampled = [text.tolist() for text in train_text_resampled]
train_images_resampled = train_images[train_labels_resampled]
train_labels_resampled = torch.tensor(train_labels_resampled)

# Reconstruct the resampled dataset
train_dataset_resampled = [(text, image, label) for text, image, label in zip(train_text_resampled, train_images_resampled, train_labels_resampled)]

# Define data loaders for train, validation, and test sets
batch_size = 8
train_loader = DataLoader(train_dataset_resampled, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the multi-modal model
class MultiModalModel(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super(MultiModalModel, self).__init__()

        # Text Model (KoElectra)
        self.text_model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.text_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

        # Image Model (MobileNetV2)
        self.image_model = models.mobilenet_v2(pretrained=True)
        self.image_model.classifier = nn.Identity()  # Remove the final fully connected layer

        # Fusion Layer
        fusion_dim = self.text_model.config.hidden_size + self.image_model.last_channel
        self.fusion_layer = nn.Linear(fusion_dim, fusion_dim)

        # Output Layer
        self.output_layer = nn.Linear(fusion_dim, num_classes)

        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text_inputs, image_inputs):
        # Text Encoding
        text_input_ids = self.text_tokenizer.batch_encode_plus(
            text_inputs,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )["input_ids"]
        text_outputs = self.text_model(input_ids=text_input_ids)[0][:, 0, :]  # Use the CLS token embedding

        # Image Encoding
        image_outputs = self.image_model.features(image_inputs)
        image_outputs = torch.nn.functional.adaptive_avg_pool2d(image_outputs, 1).reshape(image_outputs.size(0), -1)

        # Fusion
        fusion_inputs = torch.cat((text_outputs, image_outputs), dim=1)
        fused_outputs = self.fusion_layer(fusion_inputs)

        # Dropout
        fused_outputs = self.dropout(fused_outputs)

        # Output
        logits = self.output_layer(fused_outputs)

        return logits


# Initialize the multi-modal model
num_classes = 170  # Specify the number of output classes
model = MultiModalModel(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = criterion.to(device)

for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()

    for text, image, label in train_loader:
        text = text.to(device)
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        logits = model(text, image)
        loss = criterion(logits, label)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * text.size(0)

    train_loss /= len(train_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}")
